In [1]:
!pwd
!ls

/gpfs/space/home/zaliznyi
core.178524		       jupyter-zaliznyi.36319736.out
core.47197		       jupyter-zaliznyi.36323929.log
donkey.job		       jupyter-zaliznyi.36323929.out
jupyter-zaliznyi.36258456.log  miniconda3
jupyter-zaliznyi.36258456.out  Miniconda3-latest-Linux-x86_64.sh
jupyter-zaliznyi.36280898.log  mycar
jupyter-zaliznyi.36280898.out  pedestrian_detector_v2.0
jupyter-zaliznyi.36281650.log  Pedestrian.ipynb
jupyter-zaliznyi.36281650.out  PedestrianTorch.ipynb
jupyter-zaliznyi.36281900.log  projects
jupyter-zaliznyi.36281900.out  TFLiteConverter.ipynb
jupyter-zaliznyi.36319736.log


In [1]:
import os
import time
import tensorflow as tf
import numpy as np

In [2]:
model_path = "/gpfs/space/home/zaliznyi/pedestrian_detector_v2.0"
tflite_model_path = "/gpfs/space/home/zaliznyi/pedestrian_detector_v2.0.tflite"

In [16]:
# Workaround for this bug https://github.com/tensorflow/tensorflow/issues/34698
bin_path = os.path.dirname(os.sys.executable)
if 'PATH' in os.environ:
    os.environ['PATH'] += ':' + bin_path
else:
    os.environ['PATH'] = bin_path
print(bin_path)
print(os.environ['PATH'])

/gpfs/space/home/zaliznyi/miniconda3/envs/donkey/bin
/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/git-2.35.2-pkznwh5mpp74zq6p42b47pxpgbxxrgwk/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/perl-5.34.1-4ryb2w6tlh6ha54x25kkpq7l5zjq56do/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/pcre2-10.36-viek2bpe7b5ktcr3qpyl2qhipy2ybjxs/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/openssh-9.0p1-xuo4l7k2kii6nqw2x2t22jqghijrsdgi/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/curl-7.83.0-yzcvzf3nqftyqhrki2zketnom3lbg3kf/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/openssl-1.1.1o-m6uvz6ittrnhnwd3oftfljxqwkve44j7/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.0/libidn2-2.3.0-qmmoo5lcgqh4crq2cbbwqe26b6ujugzx/bin:/gpfs/space/software/cluster_software/spack/linux-centos7-x86_64/gcc-9.2.

In [17]:
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

2022-12-04 15:48:38.614056: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-12-04 15:48:38.614167: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-04 15:48:38.615084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-12-04 15:48:38.615144: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-04 15:48:38.615161: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-04 15:48:38.615174: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-04 15:48:38.615188: I t

In [3]:
model = tf.keras.models.load_model(model_path)

input_data = np.array(np.random.random_sample([1, 120, 160, 3]), dtype=np.float32)
print(input_data.shape)

start = time.perf_counter()

output_data = model(input_data)

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

2022-12-04 16:03:07.104466: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-12-04 16:03:08.316961: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:8a:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-12-04 16:03:08.365169: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-12-04 16:03:08.531801: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-12-04 16:03:08.601107: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-12-04 16:03:08.776860: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so

(1, 120, 160, 3)


2022-12-04 16:03:16.090822: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-12-04 16:03:19.063598: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


tf.Tensor([[0.3399557]], shape=(1, 1), dtype=float32)
Elapsed Time: 3.342443810775876 ms


In [4]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data.shape)

# Inference
start = time.perf_counter()

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

stop = time.perf_counter()

print(output_data)
print(f"Elapsed Time: {stop - start} ms")

(1, 120, 160, 3)
[[0.2840037]]
Elapsed Time: 0.023271989077329636 ms
